# Geometry, Axes, and Components

This notebook is the first step in the aircraft simulation and design optimization workflow. Here, you will learn how to define the geometry of an aircraft, including its main components (such as wings, fuselage, and tail surfaces) and their key parameters. 

**Important Note: If you are interested in a single code cell to run this entire example (or to use as a basis for your own work) see the code cell at the bottom of the example and uncomment all lines**

**Purpose:**
- Establish the baseline configuration for the aircraft.
- Set up the geometric parameters that can be used in subsequent analyses (aerodynamics, mass properties, free form deformation, etc.).

**What you will learn:**
- How to specify the main geometric features of an aircraft in this framework
- How to setup axis systems both disconnected and connected to geometry.
- How to define components

This notebook is intended to provide geometry setup methodology for a theoretical "Super" Cessna 172 aircraft as the first part of a multi-part tutorial series. This aircraft is a modified version of the Cessna 172, featuring six total rotors, two of which are intended as lift-only and four of which are tilt-capable. This is intended to be a theoretical novel aircraft and is solely intended to be used for demonstration purposes. The notebook will cover the following key steps:

1. **Importing Geometry**: 
   - Import a .stp file representing the aircraft geometry.

2. **Setting Up Axes**:
   - Create inertial, openvsp, wing, and pusher rotor axes from the geometry.
   - Simulate the creation of six lift rotors on the C172, including two at the wing tips (tilt-capable), two at the horizontal tail tips (tilt-capable), one in front of the wing on the fuselage (lift-only), and one in the back of the wing on the fuselage (lift-only).
3. **Defining Components**:
   - Define components for both real components from geometry (wing, fuselage, horizontal tail, vertical tail, pusher rotor) and fake components (six lift rotors). This part does not necessarily require geometry


In [ ]:
# Preliminary setup
import lsdo_function_spaces as lfs
import csdl_alpha as csdl
import numpy as np
from falco.utils.import_geometry import import_geometry
from falco import REPO_ROOT_FOLDER, Q_
from lsdo_geo.core.geometry.geometry import Geometry
from falco.core.dynamics.axis import Axis, ValidOrigins
from falco.core.dynamics.axis_lsdogeo import AxisLsdoGeo

lfs.num_workers = 1 # LSDO Function Spaces is capable of using more than 1 worker (see their documentation). For ease, and use on windows machines, we use 1.

# Unit Conversions for manual setup of geometry locations

in2m=0.0254
ft2m = 0.3048

recorder = csdl.Recorder(inline=True, expand_ops=True, debug=False)
recorder.start()


## Importing Geometry (.stp)
Let's now import the actual geometry of the aircraft. The geometry is imported from a STEP (.stp) file, which is a common format for 3D models. It is a common output of OpenVSP and other CAD software. The geometry is imported using the `import_geometry` function. This function relies on the `lsdo_geo` library, which is a part of the LSDO (Large-Scale Design Optimization) framework. 

The file name is set to "c172.stp", which is a STEP file representing the Cessna 172 aircraft. REPO_ROOT_FOLDER is a variable that points to the root folder of the flight simulator repository, but you can adjust the `file_path` parameter to point to the location of your stp file if it is not in the specified folder. The `scale` parameter is set to `in2m`, which converts the dimensions from inches to meters, as the simulation framework operates in SI units, but the original stp model was output in inches from OpenVSP. The `rotate_to_body_fixed_frame` parameter is set to `True`, which means that the geometry will be rotated to align with the body-fixed frame. This is taken as x increasing from nose to tail (longitudinal), y increasing from left to right (lateral), and z increasing from top to bottom (vertical). This means the z axis is pointing downwards, which is the standard convention for flight dynamics analysis. 


In [ ]:
sc172_geometry = import_geometry(
        file_name="c172.stp",
        file_path= REPO_ROOT_FOLDER / 'tutorials'/ 'geometries',
        refit=False,
        scale=in2m,
        rotate_to_body_fixed_frame=True
    )

# the .plot method below is used to visualize the geometry in a 3D plot. 
# View further documentation for the plot method in the lsdo_function_spaces library.
# While this will not work in a Jupyter notebook, it will work in a normal Python script.

# sc172_geometry.plot(
#     show=True)

Next we will talk about how to assign specific subsets of geometries from your .stp file to a variable. This is done using `declare_component` method from lsdo_geo's `Geometry` class. You would be interested in doing this if you want to manipulate specific parts of the geometry, such as wings, fuselage, or rotors independently instead of the entire aircraft geometry. In `declare_component`, you can specify the `function_search_names` parameter, which is a list of names that will be used to search for the components in the geometry. Depending on your output settings from OpenVSP, these names may vary. You can find the names of the components in the .stp file by opening it in a CAD software or by searching the .stp file in a text editor. The `name` parameter is used to assign a name to the component, which can be used later to reference the component in the simulation.

You will notice some searches include multiple names, such as tails searching for the horizontal stabilizer and the vertical stabilizer. This grouping, separate from the horzTail and vertTail, allows for flexibility in the geometry definition, as it allows you to reference the entire tail assembly without needing to specify each part individually for a desired manipulation while still allowing you to do so if desired. An example manipulation would be the coupled move of the horizontal and vertical stabilizers forward by the same few inches while retaining the ability to independently increase each individual tail's surface area to retain control authority.

In [5]:
wing = sc172_geometry.declare_component(function_search_names=['MainWing'], name='wing')
horzTail = sc172_geometry.declare_component(function_search_names=['HTail'], name='horzTail')
vertTail = sc172_geometry.declare_component(function_search_names=['VerticalTail'], name='vertTail')
fuselage = sc172_geometry.declare_component(function_search_names=['Fuselag'], name='fuselage')

tails = sc172_geometry.declare_component(function_search_names=['Htail, VerticalTail'], name='tails')

Notice in the above section there is no definition of the traditional propellers on the front of the fuselage of the Cessna 172. Even though it was included in the original geometry of the .stp file, we choose not to import it because we are not interested in adding the forward propeller to our super cessna aircraft. This, and other propulsors, will be added later in the tutorial as components with no assigned geometry.

## Geometry Point Parameterization and Evaluation
We will next focus on parameterization of specific points on each of these geometries. This is done by guessing a point nearest to the desired point and then using the `project` method to find the closest point on the geometry. We will then use the `evaluate` method to get the function evaluated point. Guesses are taken from OpenVSP or from the CAD model. Notice each point starts with a guess, a projection on a subset of geometry, then an evaluation on the full geometry. 

In [6]:
# Wing Parameterization
# TODO: Fill guesses
wing_le_left_guess = np.array([])*ft2m
wing_le_left_parametric = wing.project(wing_le_left_guess, plot=False)
wing_le_left = sc172_geometry.evaluate(wing_le_left_parametric)

wing_le_right_guess = np.array([])*ft2m
wing_le_right_parametric = wing.project(wing_le_right_guess, plot=False)
wing_le_right = sc172_geometry.evaluate(wing_le_right_parametric)

wing_le_center_guess = np.array([])*ft2m
wing_le_center_parametric = wing.project(wing_le_center_guess, plot=False)
wing_le_center = sc172_geometry.evaluate(wing_le_center_parametric)

wing_te_left_guess = np.array([])*ft2m
wing_te_left_parametric = wing.project(wing_te_left_guess, plot=False)
wing_te_left = sc172_geometry.evaluate(wing_te_left_parametric)

wing_te_right_guess = np.array([])*ft2m
wing_te_right_parametric = wing.project(wing_te_right_guess, plot=False)
wing_te_right = sc172_geometry.evaluate(wing_te_right_parametric)

wing_te_center_guess = np.array([])*ft2m
wing_te_center_parametric = wing.project(wing_te_center_guess, plot=False)
wing_te_center = sc172_geometry.evaluate(wing_te_center_parametric)

ValueError: operands could not be broadcast together with shapes (3,) (0,) 

In [ ]:
# Horizontal Tail Parameterization
# TODO: Horizontal Tail

In [ ]:
# Vertical Tail Parameterization
# TODO: Vertical Tail

In [ ]:
# Fuselage Parameterization
# TODO: Fuselage

## Axis System Setup
Next we will discuss the setup of various axes systems used in the framework. These can be tied to geometry if desired, but are not required to be. If they are tied to geometry they will be setup using the `AxisLsdoGeo` function instead of `Axis`. The axes systems are used to define the orientation and position of the aircraft in the simulation. Axis systems are defined relative to others, with the two base axis origins being the Inertial and the OpenVSP origins. 

The most important axis to setup for your model is the flight dynamics body fixed axis, which can be anywhere in your model. For ours, we will set it to be with respect to the inertial axis system with no translation offset. This body fixed axis system is what all load solvers translate loads to.



In [ ]:
# Inertial Axis
inertial_axis = Axis(
    name='Inertial Axis',
    origin=ValidOrigins.Inertial.value
)

# OpenVSP Axis
openvsp_axis = Axis(
    name='OpenVSP Axis',
    x=Q_(0, 'm'),
    y=Q_(0, 'm'),
    z=Q_(0, 'm'),
    origin=ValidOrigins.OpenVSP.value
)

# Flight Dynamics Body Fixed Axis
fd_axis = Axis(
        name='Flight Dynamics Body Fixed Axis',
        x=Q_(0, 'ft'),
        y=Q_(0, 'ft'),
        z=Q_(0, 'ft'),  
        phi=Q_(0, 'deg'),
        theta=Q_(0, 'deg'),
        psi=Q_(0, 'deg'),
        sequence=np.array([3, 2, 1]),
        reference=inertial_axis,
        origin=ValidOrigins.Inertial.value
    )

# AxisLsdoGeo for Geometry
#TODO: Setup geometry axes systems

## Components
Finally, we will define the components of the aircraft. Components are used to define the main parts of the aircraft, such as wings, fuselage, and tail surfaces. Components can be defined with or without geometry. In this case, we will define the components with geometry for the real components (wing, fuselage, horizontal tail, vertical tail, pusher rotor) and without geometry for the "fake" components (six rotors). 

In [ ]:
#TODO: Define Components